In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models

def conv_module(x, filters, kernel_size, strides):
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    return x

def inception_module(x, filters):
    conv1x1 = conv_module(x, filters[0], (1, 1), 1)

    conv3x3 = conv_module(x, filters[1], (1, 1), 1)
    conv3x3 = conv_module(conv3x3, filters[2], (3, 3), 1)

    conv5x5 = conv_module(x, filters[3], (1, 1), 1)
    conv5x5 = conv_module(conv5x5, filters[4], (5, 5), 1)

    maxpool = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    maxpool = conv_module(maxpool, filters[5], (1, 1), 1)

    return layers.concatenate([conv1x1, conv3x3, conv5x5, maxpool], axis=-1)

def downsample_module(x, filters):
    x = layers.Conv2D(filters, (3, 3), strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    return x

def build_inception_model():
    input_shape = (32, 32, 3)
    num_classes = 10

    inputs = tf.keras.Input(shape=input_shape)

    x = conv_module(inputs, 64, (7, 7), 2)
    x = layers.MaxPooling2D((3, 3), strides=2, padding='same')(x)

    x = conv_module(x, 192, (3, 3), 1)
    x = layers.MaxPooling2D((3, 3), strides=2, padding='same')(x)

    x = inception_module(x, [64, 128, 128, 32, 32, 32])
    x = inception_module(x, [128, 192, 96, 64, 64, 64])

    x = downsample_module(x, 128)

    x = inception_module(x, [192, 208, 96, 64, 64, 64])
    x = inception_module(x, [160, 224, 112, 64, 64, 64])
    x = inception_module(x, [128, 256, 128, 64, 64, 64])
    x = inception_module(x, [112, 288, 144, 64, 64, 64])
    x = inception_module(x, [256, 320, 160, 128, 128, 128])

    x = downsample_module(x, 256)

    x = inception_module(x, [256, 320, 160, 128, 128, 128])
    x = inception_module(x, [384, 384, 192, 128, 128, 128])

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.4)(x)

    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)

    return model

# Create the model
model = build_inception_model()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 16, 16, 64)           9472      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 16, 16, 64)           256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 8, 8, 64)             0         ['batch_normalization[0][0

In [13]:
import tensorflow as tf
from tensorflow.keras import datasets, utils
from sklearn.model_selection import train_test_split

# Load and preprocess the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, valid_images, train_labels, valid_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Normalize pixel values to be between 0 and 1
train_images, valid_images, test_images = train_images / 255.0, valid_images / 255.0, test_images / 255.0

# Convert class vectors to binary class matrices
train_labels = utils.to_categorical(train_labels, 10)
valid_labels = utils.to_categorical(valid_labels, 10)
test_labels = utils.to_categorical(test_labels, 10)

# Build the Inception model
model = build_inception_model()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(valid_images, valid_labels))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc * 100:.2f}%')



Epoch 1/10
Cause: Unable to locate the source code of <function Model.make_train_function.<locals>.train_function at 0x0000019BEFCFC040>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function Model.make_train_function.<locals>.train_function at 0x0000019BEFCFC040>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. O

In [14]:
import pickle

test_predictions = model.predict(test_images)

# Save the predictions to a pickle file
pickle_filename = 'test_predictions.pkl'
with open(pickle_filename, 'wb') as file:
    pickle.dump(test_predictions, file)

print(f'Test predictions saved to {pickle_filename}')

Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.predict_function at 0x0000019BEEA02200>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.predict_function at 0x0000019BEEA02200>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Orig

In [16]:
# Load the saved predictions from the pickle file
with open(pickle_filename, 'rb') as file:
    loaded_predictions = pickle.load(file)

print("Loaded Predictions:")
print(loaded_predictions)
    

Loaded Predictions:
[[2.1296931e-02 5.3352462e-03 4.0131848e-02 ... 1.1561748e-02
  5.1927110e-03 1.5192354e-03]
 [4.0948708e-03 3.3355500e-03 3.5668683e-04 ... 8.9396290e-06
  9.9029571e-01 9.1808010e-04]
 [1.0217691e-01 3.8474366e-02 5.2353796e-02 ... 3.8636526e-01
  4.0788703e-02 3.8412195e-02]
 ...
 [1.7652182e-02 5.6356224e-03 2.6777057e-02 ... 7.1822703e-02
  3.9921865e-02 6.0250619e-03]
 [7.5272724e-02 3.7525590e-03 2.2008026e-02 ... 6.8383120e-02
  1.3921110e-02 5.7404949e-03]
 [8.9849031e-04 2.6850455e-04 9.5759571e-04 ... 6.8417621e-01
  2.5986956e-04 9.1287756e-04]]


In [17]:
# Check if the current test accuracy is higher than the previously stored maximum
max_accuracy_filename = 'max_accuracy.pkl'
try:
    with open(max_accuracy_filename, 'rb') as file:
        max_accuracy = pickle.load(file)
    if test_acc > max_accuracy:
        max_accuracy = test_acc
        with open(max_accuracy_filename, 'wb') as file:
            pickle.dump(max_accuracy, file)
except FileNotFoundError:
    # If the file doesn't exist, create it and store the current test accuracy
    max_accuracy = test_acc
    with open(max_accuracy_filename, 'wb') as file:
        pickle.dump(max_accuracy, file)

# Print the maximum accuracy
print(f'Maximum Test Accuracy: {max_accuracy * 100:.2f}%')

Maximum Test Accuracy: 60.49%
